# AITA Data Loading
This file will be dedicated pushing an initial load of reddit data to Google big query. Further data changes will occur later in other ipynb files like *AITA_GCPDataModifications.ipynb* 

**NOTE:** Do not run this file again

In [4]:
import sys
import importlib
import pandas as pd
import numpy as np
from pathlib import Path
from dotenv import load_dotenv
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

In [5]:
sys.path.append("..")

# load enviornment variables for praw to work later
load_dotenv(dotenv_path=Path("../settings.env"))

True

In [6]:
client = bigquery.Client()

In [56]:
# helper function to check if table exists
def gcp_table_exists(client: bigquery.Client, table_id: str):
    try:
        client.get_table(table_id)  # Make an API request.
        return True
    except NotFound:
        return False

## Create Dataset
Using Google Bigquery, create datasets for the data that is about to be loaded. The dataset will be called **AITA_dataset**

In [57]:
PROJ_NAME = "bonion"
DATASET_NAME = "AITA_dataset"

#### Get all dataset ID's
To prevent overwriting pre-existing datasets, get all dataset id's to check before any creation step.

In [58]:
# Get all datasets 
datasets = list(client.list_datasets())  # Make an API request.
dataset_ids = [dataset.dataset_id for dataset in datasets]
project = client.project

if datasets:
    print("Datasets in project {}:".format(project))
    for dataset in datasets:
        print("\t{}".format(dataset.dataset_id))
else:
    print("{} project does not contain any datasets.".format(project))

Datasets in project bonion:
	AITA_dataset


In [59]:
# create dataset
if DATASET_NAME not in dataset_ids:
    dataset = bigquery.Dataset("{}.{}".format(PROJ_NAME, DATASET_NAME))
    dataset.location = "US"

    # send dataset to API for completion
    initial_post_dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
    print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

## Post Table
The initial post for each of the comments will need to be stored to reference its content which dictated the reply.

In [60]:
# table id definitions
post_table_id = "{}.{}.post_table".format(PROJ_NAME, DATASET_NAME)
comment_table_id = "{}.{}.comment_table".format(PROJ_NAME, DATASET_NAME)
reply_table_id = "{}.{}.reply_table".format(PROJ_NAME, DATASET_NAME)

#### Get all Table ID's
To prevent overwriting pre-existing tables, get all table id's to check before any creation step.

In [61]:
tables = list(client.list_tables(DATASET_NAME))  # Make an API request.
table_ids = [table.table_id for table in tables]

print("Tables contained in '{}':".format(DATASET_NAME))
for table in tables:
    print("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))

Tables contained in 'AITA_dataset':
bonion.AITA_dataset.post_table


In [10]:
post_table_schema = [
    bigquery.SchemaField("reddit_post_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("post_title", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("post_self_text", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("upvotes", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("num_responses", "INTEGER", mode="REQUIRED")
]

if not gcp_table_exists(client, table_id=post_table_id): 
    post_table = bigquery.Table(post_table_id, schema=post_table_schema)
    post_table.description = """
        A table which holds popular posts from the subreddit r/AITA
    """

    table = client.create_table(post_table)
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

## Comment Table
This table holds all of the responses for each of the posts in the post table.

In [62]:
comment_table_schema = [
    bigquery.SchemaField("comment_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("parent_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("content", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("upvotes", "INTEGER", mode="REQUIRED")
]

if not gcp_table_exists(client, table_id=comment_table_id): 
    post_reply_table = bigquery.Table(comment_table_id, schema=comment_table_schema)
    post_reply_table.description = """
        A table which holds the most popular replys to saved posts from the subreddit r/AITA
    """

    table = client.create_table(post_reply_table)
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

Created table bonion.AITA_dataset.comment_table


## Reply Table
This table will hold replys to the initial post reply. This will be used for also dictating the performance of a reply. Because there can be infinite replies to any given post or reply, each post reply id will be limited to some number of replies. These replies will likley be based off of the same metrics as the original post for "quality". 

In [63]:
reply_table_schema = [
    bigquery.SchemaField("reply_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("parent_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("content", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("upvotes", "INTEGER", mode="REQUIRED")
]   

if not gcp_table_exists(client, table_id=reply_table_id): 
    post_reply_top_children_table = bigquery.Table(reply_table_id, schema=reply_table_schema)
    post_reply_top_children_table.description = """
        This table will hold replys to the initial post reply. 
        This will be used for also dictating the performance of a reply. 
        Because there can be infinite replies to any given post or reply, 
        each post reply id will be limited to some number of replies. 
        These replies will likley be based off of the same metrics as 
        the original post for "quality". 
    """

    table = client.create_table(post_reply_top_children_table)
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

Created table bonion.AITA_dataset.reply_table


## Collect Data
Using my PrawInstance Object, I will be collecting the top posts from the subreddit. I will be storing this data in google bigquery.

In [64]:
import praw_instance
importlib.reload(praw_instance)

<module 'praw_instance' from '/home/cstainsby/class/dataProj/bonion/src/notebooks/../praw_instance.py'>

In [65]:
subreddit_name = "amitheasshole"

In [66]:
praw_inst = praw_instance.PrawInstance()

### Store Top 1000 Posts

In [16]:
top_10000_posts = praw_instance.get_top_by_subreddit(praw_inst, subreddit_name, limit=1000)

KeyboardInterrupt: 

In [ ]:
# store the top 1000 posts into gcp
# convert data into json rows 
post_table_df = praw_instance.post_dict_to_df(top_10000_posts)

post_table_df.head(10)

In [ ]:
# cast table columns to type
post_table_df["reddit_post_id"] = post_table_df["reddit_post_id"].astype(str)
post_table_df["post_title"] = post_table_df["post_title"].astype(str)
post_table_df["post_self_text"] = post_table_df["post_self_text"].astype(str)
post_table_df["upvotes"] = post_table_df["upvotes"].astype(int)
post_table_df["num_responses"] = post_table_df["num_responses"].astype(int)
post_table_df.info()

#### Push Data

In [1]:
import pandas_gbq

In [2]:
pandas_gbq.to_gbq(post_table_df, post_table_id, project_id=PROJ_NAME)

NameError: name 'post_table_df' is not defined

### Get Top Comments and Replies for the top posts

In [67]:
# get the top post ids

query = """
    SELECT reddit_post_id
    FROM {}
""".format(post_table_id)

post_table_ids_df = pd.read_gbq(query, project_id=PROJ_NAME)

post_table_ids_df.head(4)

,reddit_post_id
0,jyk2ac
1,ukzctc
2,u90414
3,socrlh


In [80]:
comment_and_reply_dict_list = []

# I will batch the groups of comments I'm grabbing
# so far I have pulled indices 0 thru 300
start_index = 51 
stop_index = 300

comment_limit = 10
reply_limit = 5

print("Pulling {} comments and {} replies for each comment:".format(comment_limit, reply_limit))
for index, row in post_table_ids_df.iterrows():
    if index >= start_index and index <= stop_index: 
        print("\tOn index {} of {}".format(index - start_index, stop_index - start_index))
        comment_and_reply_dict = praw_instance.get_top_comments_and_top_replies_by_post_id(
            praw_inst,
            row["reddit_post_id"],
            comment_limit=comment_limit,
            reply_limit=reply_limit
        )

        comment_and_reply_dict_list.append(comment_and_reply_dict)

Pulling 10 comments and 5 replies for each comment:
	On index 0 of 249
	On index 1 of 249
	On index 2 of 249
	On index 3 of 249
	On index 4 of 249
	On index 5 of 249
	On index 6 of 249
	On index 7 of 249
	On index 8 of 249
	On index 9 of 249
	On index 10 of 249
	On index 11 of 249
	On index 12 of 249
	On index 13 of 249
	On index 14 of 249
	On index 15 of 249
	On index 16 of 249
	On index 17 of 249
	On index 18 of 249
	On index 19 of 249
	On index 20 of 249
	On index 21 of 249
	On index 22 of 249
	On index 23 of 249
	On index 24 of 249
	On index 25 of 249
	On index 26 of 249
	On index 27 of 249
	On index 28 of 249
	On index 29 of 249
	On index 30 of 249
	On index 31 of 249
	On index 32 of 249
	On index 33 of 249
	On index 34 of 249
	On index 35 of 249
	On index 36 of 249
	On index 37 of 249
	On index 38 of 249
	On index 39 of 249
	On index 40 of 249
	On index 41 of 249
	On index 42 of 249
	On index 43 of 249
	On index 44 of 249
	On index 45 of 249
	On index 46 of 249
	On index 47 of 24

In [81]:
# convert the data to a dataframe
full_comment_list, full_reply_list = [], []
for post_i, comment_and_replies_dict in enumerate(comment_and_reply_dict_list):
    comment_df_at_post_i, reply_df_at_post_i =  praw_instance.comment_and_reply_dict_to_df(comment_and_replies_dict)

    # store df's in an intermediate list before combining into a df
    full_comment_list.append(comment_df_at_post_i)
    full_reply_list.append(reply_df_at_post_i)

comment_table_df = pd.concat(full_comment_list, axis=0, ignore_index=True)
reply_table_df = pd.concat(full_reply_list, axis=0, ignore_index=True)

In [82]:
comment_table_df["comment_id"] = comment_table_df["comment_id"].astype(str)
comment_table_df["parent_id"] = comment_table_df["parent_id"].astype(str)
comment_table_df["content"] = comment_table_df["content"].astype(str)
comment_table_df["upvotes"] = comment_table_df["upvotes"].astype(int)

print("Comment table length", len(comment_table_df))
print(comment_table_df.info())
comment_table_df.head()

Comment table length 2377
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2377 entries, 0 to 2376
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   comment_id  2377 non-null   object
 1   parent_id   2377 non-null   object
 2   content     2377 non-null   object
 3   upvotes     2377 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 74.4+ KB
None


,comment_id,parent_id,content,upvotes
0,g1lb1ef,t3_ia7s4c,"NTA\n\nYou said he’d eat it or something, the ...",2853
1,g1lg7ks,t3_ia7s4c,"NTA- What did they think was going to happen, ...",597
2,g1laxsy,t3_ia7s4c,"NTA. I mean, you told him not to give it to he...",19301
3,g1lave0,t3_ia7s4c,"NTA. You hand a toddler a toy, not actual money.",3937
4,g1laydm,t3_ia7s4c,Lmaooo NTA-why give money in bare cash to a 1-...,709


In [83]:
reply_table_df["reply_id"] = reply_table_df["reply_id"].astype(str)
reply_table_df["parent_id"] = reply_table_df["parent_id"].astype(str)
reply_table_df["content"] = reply_table_df["content"].astype(str)
reply_table_df["upvotes"] = reply_table_df["upvotes"].astype(int)

print("Reply table length", len(reply_table_df))
print(reply_table_df.info())
reply_table_df.head()

Reply table length 8118
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8118 entries, 0 to 8117
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   reply_id   8118 non-null   object
 1   parent_id  8118 non-null   object
 2   content    8118 non-null   object
 3   upvotes    8118 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 253.8+ KB
None


,reply_id,parent_id,content,upvotes
0,g1myvit,t1_g1lb1ef,5 seconds ago*,354
1,g1nj0tp,t1_g1lb1ef,"Relative has a screw loose, its just plain stupid",51
2,g1oh6th,t1_g1lb1ef,this is truly the year of r/LeopardsAteMyFace,6
3,g1m6iz6,t1_g1lg7ks,that's some unrealistic toddler standards my k...,465
4,g1mvoxy,t1_g1lg7ks,Invest in babycoin,39


### Push Comments and Replies 

In [84]:
pandas_gbq.to_gbq(comment_table_df, comment_table_id, project_id=PROJ_NAME, if_exists="append")
pandas_gbq.to_gbq(reply_table_df, reply_table_id, project_id=PROJ_NAME, if_exists="append")

100%|██████████| 1/1 [00:00<00:00, 5121.25it/s]
